# 文件说明

- `OSM data`：是指从OpenStreetMap获取的原始地理数据。这些数据通常包括建筑物、道路、河流等的地理信息。OSM数据一般使用WGS 84（即全球最常用的地理坐标系）来表示位置，也就是以经纬度坐标为主。
- `Projected data`：是对地理数据进行投影变换后的数据，转换的结果是将原始的经纬度坐标系转换为一个平面坐标系（如UTM坐标系）。这种投影可以更方便地在本地尺度上进行距离计算、空间分析等操作。投影数据是以二维坐标表示的，单位通常是米或千米，而不是经纬度。

# 文件组成

- `NYC.shp`：这是Shapefile的主文件，包含实际的几何数据，比如多边形、点、线等，用来表示地理特征（在这里是纽约市的边界）。
- `NYC.dbf`：这是Shapefile的属性表文件，包含与几何对象相关的属性数据（例如地名、人口密度等）。
- `NYC.prj`：这是投影文件，包含描述数据投影信息的文本，定义了坐标系统和投影方式。这个文件表明数据经过投影转换。
- `NYC.shx`：这是Shapefile的索引文件，帮助加快几何体的读取速度。
- `NYC.cpg` 和 `NYC.sbn`, `NYC.sbx`：这些是Shapefile的编码、空间索引文件，提供额外的元数据或索引支持。
- `NYC.shp.xml`：这是Shapefile的元数据文件，包含该文件的一些详细描述。

In [8]:
import fiona

NYC_shapefile_list= {
    "boundary": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/boundary/NYC.shp",
    "building": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/building/building.shp",
    "landuse": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/landuse/landuse.shp",
    "poi": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/poi/poi.shp",
    "population": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/population/population.shp",
    "region": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/region/region.shp",
    "segmentation": "/hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/segmentation/segmentation.shp"
}

def read_shapefile(shapefile_name, shapefile_path: str, max_features=5):
    """读取指定的Shapefile，并打印前几个要素"""
    try:
        with fiona.open(shapefile_path, 'r') as shp:
            print(f"\nReading {shapefile_name} shapefile from {shapefile_path}")
            print(f"Total features: {len(shp)}")
            
            # 打印前 max_features 个 feature 的几何和属性信息
            for i, feature in enumerate(shp):
                if i >= max_features:
                    print(f"... Only showing first {max_features} features.")
                    break
                print(f"Feature {i + 1}:")
                print(f"  Geometry: {feature['geometry']}")
                print(f"  Properties: {feature['properties']}")
                
    except Exception as e:
        print(f"Error reading {shapefile_name} shapefile: {e}")

# 遍历并读取所有Shapefile
for shapefile_name, shapefile_path in NYC_shapefile_list.items():
    read_shapefile(shapefile_name, shapefile_path)


Reading boundary shapefile from /hpc2hdd/home/szhong691/zsr/projects/RegionDCL/data/projected/NYC/boundary/NYC.shp
Total features: 5
Feature 1:
  Geometry: fiona.Geometry(coordinates=[[[(295722.7399000004, 44392.17639999837), ...]], ...], type='MultiPolygon')
  Properties: fiona.Properties(boro_code=5.0, boro_name='Staten Island', shape_leng=325924.002076, Shape_Le_1=99339.1208826, Shape_Area=150841072.397)
Feature 2:
  Geometry: fiona.Geometry(coordinates=[[[(308708.70380000025, 69868.91479999945), ...]], ...], type='MultiPolygon')
  Properties: fiona.Properties(boro_code=2.0, boro_name='Bronx', shape_leng=463277.240478, Shape_Le_1=141203.943416, Shape_Area=110294116.075)
Feature 3:
  Geometry: fiona.Geometry(coordinates=[[[(299076.16640000045, 57502.25530000031), ...]], ...], type='MultiPolygon')
  Properties: fiona.Properties(boro_code=1.0, boro_name='Manhattan', shape_leng=359103.151368, Shape_Le_1=109454.165251, Shape_Area=59142823.5621)
Feature 4:
  Geometry: fiona.Geometry(coor

这些属性参数是从不同的 **Shapefile** 中读取的属性信息，它们为每个地理要素（如多边形或点）提供了相关的**非几何数据**，通常包括**行政区信息**、**建筑属性**、**土地使用情况**、**人口数据**、**兴趣点**等。这些属性数据是通过 `fiona` 从 `.dbf` 文件（与 `.shp` 文件配套的数据库文件）中提取的。我们逐一解释这些属性参数。

### 1. **Boundary Shapefile (`boundary`)**
这个文件表示纽约市不同行政区（borough）的边界信息。相关属性参数说明如下：

- **`boro_code`**: 代表行政区代码。纽约市有五个行政区：
  - 1: 曼哈顿 (Manhattan)
  - 2: 布朗克斯 (Bronx)
  - 3: 布鲁克林 (Brooklyn)
  - 4: 皇后区 (Queens)
  - 5: 史泰登岛 (Staten Island)
  
- **`boro_name`**: 行政区名称，对应上面的代码。
  
- **`shape_leng`**: 边界的总长度（通常以米为单位）。
  
- **`Shape_Area`**: 行政区的总面积（通常以平方米为单位）。

### 2. **Building Shapefile (`building`)**
这个文件表示纽约市的建筑信息，通常是从 OpenStreetMap 数据中提取的。相关属性参数说明如下：

- **`osm_id`**: OpenStreetMap（OSM）中的对象ID，唯一标识该建筑物。
  
- **`code`**: 建筑物的分类代码（通常是内部代码）。
  
- **`fclass`**: 建筑物的功能类别，例如 `building` 表示这是一个建筑物，其他可能的值包括不同类型的建筑物，如 `church`、`school` 等。
  
- **`name`**: 建筑物的名称。如果没有名称，字段为空。
  
- **`type`**: 建筑物的类型，例如 `residential` 表示住宅建筑。

- **`Shape_Leng`**: 建筑物的周长。
  
- **`Shape_Area`**: 建筑物的占地面积。

### 3. **Land Use Shapefile (`landuse`)**
这个文件表示纽约市的土地使用信息。土地使用类型通常由政府规划机构提供。相关属性参数说明如下：

- **`Borough`**: 行政区缩写，例如 `MN` 表示曼哈顿。
  
- **`Block`**: 街区代码。
  
- **`Lot`**: 街区中的地块编号。
  
- **`Address`**: 该地块的地址信息。
  
- **`LandUse`**: 土地使用类型，例如 `07` 可能表示商业用地，其他可能值包括住宅用地、工业用地等。
  
- **`OwnerType`**: 土地所有者的类型，例如 `C` 可能表示公司（Corporation）。
  
- **`OwnerName`**: 土地所有者的名称。
  
- **`LotArea`**: 地块的总面积（通常以平方英尺为单位）。
  
- **`YearBuilt`**: 建筑物的建造年份。
  
- **`UnitsRes`**: 住宅单元的数量。
  
- **`AssessLand`**: 该地块的土地评估价值。
  
- **`AssessTot`**: 该地块的总评估价值（包括建筑物）。

### 4. **Points of Interest (POI) Shapefile (`poi`)**
这个文件表示纽约市的兴趣点（POI），例如公园、商店、旅游景点等。相关属性参数说明如下：

- **`osm_id`**: OpenStreetMap 中该兴趣点的唯一ID。
  
- **`code`**: 兴趣点的分类代码，表示该 POI 的类别。
  
- **`fclass`**: 功能类别，例如 `bench` 表示长凳，`attraction` 表示旅游景点，`camera_surveillance` 表示监控摄像头，`drinking_water` 表示饮水点。
  
- **`name`**: 兴趣点的名称。如果没有名字，字段为空。

### 5. **Region Shapefile (`region`)**
这个文件表示纽约市的行政区划，可能是更小的区域或社区，如街区、街道等。相关属性参数说明如下：

- **`CTLabel`**: 区块编号标签。
  
- **`BoroCode`**: 行政区代码，与 `boundary` 文件中的 `boro_code` 相同。
  
- **`BoroName`**: 行政区名称。
  
- **`NTAName`**: 地区名称（Neighborhood Tabulation Area, NTA），表示纽约市的社区分区。
  
- **`Shape_Leng`**: 区域边界的长度。
  
- **`Shape_Area`**: 区域的面积。

### 6. **Segmentation Shapefile (`segmentation`)**
这个文件表示纽约市的某些地理特征的分割信息。它可能是更细粒度的分区，例如某些具体的建筑区或子区域。相关属性参数说明如下：

- **`boro_code`**: 行政区代码，与其他文件中的 `boro_code` 相同。
  
- **`boro_name`**: 行政区名称。
  
- **`shape_leng`**: 该区域的边界长度。
  
- **`Shape_Area`**: 区域的面积。
  
- **`ORIG_FID`**: 原始要素的唯一ID（来自原始数据集）。
  
- **`Shape_Le_1`** 和 **`Shape_Le_2`**: 这些字段通常是与几何形状相关的统计信息，如长度、面积等。

---

### 总结：
这些 Shapefile 的属性数据反映了不同类型的地理信息。例如：
- **`boundary`** 文件为你提供了纽约五个行政区的边界信息和基本属性。
- **`building`** 文件列出了建筑物的几何形状、分类和属性。
- **`landuse`** 文件详细描述了土地使用情况、所有权信息以及评估价值。
- **`poi`** 文件描述了城市中的各种兴趣点。
- **`region`** 和 **`segmentation`** 文件则描述了更细粒度的区域划分，提供了地理区域的标签和属性。

这些属性可以用于进一步的空间分析、数据可视化以及城市规划等领域。